In [ ]:
import json
import pandas as pd
import re
from IPython.utils.text import marquee

pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 1000

In [ ]:
def ToExcel(df, filename):
    with pd.ExcelWriter(filename, engine='xlsxwriter', engine_kwargs={'options':{'strings_to_numbers': True}}) as writer:
        df.to_excel(writer, sheet_name='Sheet1', startrow=1, header=False, index=False)
        (max_row, max_col) = df.shape
        column_settings = [{'header' : c} for c in df.columns]
        writer.sheets['Sheet1'].add_table(0, 0, max_row, max_col-1, {'columns' : column_settings})

# Read MinnesotaStateFair JSON file

In [ ]:
with open('MinnesotaStateFair.json', 'rb') as f:
    js = json.loads(f.read())

In [ ]:
sorted(list(js.keys()))

In [ ]:
tables = [
    'days', 
    'entertainmentCategories', 'entertainmentEntertainers', 'entertainmentEvents',
    'locations', 'newAttractions',     
    'sponsors', 'sponsorLocations',
    'vendors', 'vendorCategories', 'vendorNewThisYear', 
]

for table in tables:
    df = pd.DataFrame().from_dict(js[table], orient='columns')
    df.to_csv(f'./csv/{table}.csv', index=False)
    ToExcel(df, (f'./excel/{table}.xlsx'))

# Events

In [ ]:
df_entertainment_events = pd.read_csv('./csv/entertainmentEvents.csv', parse_dates=['datetime_start', 'datetime_end'])#, dtype={'vendor_id': 'object'})
display(df_entertainment_events.head(1))

In [ ]:
df_entertainment_events.isna().sum()

In [ ]:
df_entertainment_events[df_entertainment_events['latitude'].isna()].head()

In [ ]:
for col in ['type', 'all_day_flag', 'name']:
    print(marquee(col))
    display(df_entertainment_events[col].value_counts().head(10))

In [ ]:
df_entertainment_events[df_entertainment_events['all_day_flag'] == 1]

In [ ]:
len(df_entertainment_events['name'].unique())

In [ ]:
df_entertainment_events.head(1)

In [ ]:
df_entertainment_events['id_event'].value_counts()

In [ ]:
df_events = (
    df_entertainment_events
    .assign(date = lambda df_: df_['datetime_start'].dt.date)
    .assign(time = lambda df_: df_['datetime_start'].dt.time)
    .drop(columns=['id_entertainer', 'slug', 'tour_name', 'billing_order', 'seating_description', 'type', 'location_path']) #, 'datetime_start', 'datetime_end'])
    .rename(columns={'id_event': 'id', 'all_day_flag': 'isAllDay'})
    .astype({'id': 'object', 'id_location': 'object', 'isAllDay': 'boolean'})
)
df_events.dtypes

In [ ]:
df_events.groupby(['name', 'date'])['time'].agg(times=list).reset_index()

In [ ]:
df = df_events.query('name == "Timberworks Lumberjack Show"').sort_values('datetime_start')
df

In [ ]:
len(df_events.groupby(['name', 'location'])['id'].count().reset_index())

In [ ]:
len(df_events.groupby(['name', 'location'])['id'].count().reset_index().drop_duplicates(subset=['name']))

In [ ]:
print(df.groupby(['name', 'date'])['time'].agg(times=list).reset_index().to_json(orient='records', indent=2))


In [ ]:
print(dd.set_index('name').to_json(orient='table', indent=2))

In [ ]:
df_events = (
    df_entertainment_events
    .query('name == "Timberworks Lumberjack Show"')
    .drop(columns=['id_entertainer', 'slug', 'tour_name', 'billing_order', 'seating_description', 'type', 'location_path'])
    .rename(columns={'id_event': 'id', 'all_day_flag': 'isAllDay'})
    .astype({'id': 'object', 'id_location': 'object', 'isAllDay': 'boolean'})
    .assign(date = lambda df_:df_['datetime_start'].dt.date)
    .assign(time = lambda df_:df_['datetime_start'].dt.time)
)

#print(df_events.shape)
#display(df_events.sample(20))
    

df_events = (
    df_events
#    .merge(df_events.groupby(['name'])['datetime_start'].agg(datetime_starts=list).reset_index(), how='left', on='name')
    .merge(df_events.groupby(['name', 'date'])['time'].agg(times=list).reset_index(), how='left')
#    .drop_duplicates(subset=['name'])
    .drop(columns=['datetime_start', 'datetime_end', 'time'])
)

#print(df_events.shape)

#display(df_events.sample(20))
#display(df_events.dtypes)


location_columns = ['location', 'latitude', 'longitude']
df_event_locations = ( 
    df_events[['id_location'] + location_columns]
    .rename(columns={'id_location' : 'id'})
    .drop_duplicates(subset=['id'])
)
    
                               
df_events = (
    df_events.drop(columns=location_columns)
)

#display(df_events.head())
#print(df_events.shape)

#display(df_event_locations.head())
#print(df_event_locations.shape)

df_events

In [ ]:
df_events.groupby(['name', 'date'])['times'].agg(ttt=list).reset_index().sample(50)

In [ ]:
df_events

In [ ]:
df_events.name.value_counts()

In [ ]:
df_events[df_events['location'] == "O'Gara's at the Fair"]

# Vendors New This Year

In [ ]:
df_new_vendors = pd.read_csv('./csv/vendorNewThisYear.csv', dtype={'vendor_id': 'object'})
display(df_new_vendors.head(1))

columns = ['vendor_id', 'name', 'excerpt', 'description']
df_new_vendors = df_new_vendors[columns]
df_new_vendors.head(1)

In [ ]:
df_new_vendors.dtypes

In [ ]:
df_new_vendors.columns

In [ ]:
df_new_vendors[df_new_vendors['excerpt'] != df_new_vendors['description']]

# Vendor Categories

In [ ]:
df_vendor_categories = pd.read_csv('./csv/vendorCategories.csv')
df_vendor_categories

# Vendors

In [ ]:
df_vendors = pd.read_csv('./csv/vendors.csv', dtype={'id':'object'})
df_vendors.query('name == "Mazda"')

In [ ]:
df_vendors['name'].value_counts().head()

In [ ]:
print(df_vendors.shape)
print(df_vendors.dtypes)

In [ ]:
len(df_vendors['name'].unique())

In [ ]:
# description - not useful
df_vendors['description'].unique()[:20]

In [ ]:
# new_this_year - not useful
df_vendors['new_this_year'].unique()

In [ ]:
# promotions - not really needed
df_vendors['promotions'].unique()[:20]

In [ ]:
cols = ['id', 'name', 'alcoholic_beverages', 'license_text', 'keywords', 'directions', 'latitude', 'longitude', 'is_food', 'categories']
df_vendors_filter_columns = (
    df_vendors
    .assign(name = lambda df_:df_['name'].str.strip())
    .loc[:, cols]
    .fillna('')
)

print(df_vendors_filter_columns.shape)
df_vendors_filter_columns.head(5)

In [ ]:
df_vendors_filter_columns.dtypes

In [ ]:
# 0 = not food, don't know what the other values represent
df_vendors_filter_columns['is_food'].value_counts()

In [ ]:
df_vendors_filter_columns.query('is_food == 5')

In [ ]:
df_vendors_filter_columns.query('is_food == 2').head(10)

In [ ]:
df_vendors_filter_columns['categories'].value_counts()

In [ ]:
# category 41 : New Food/Flavor
mask = df_vendors_filter_columns['categories'].apply(lambda x: '41' in x)
print(df_vendors_filter_columns[mask].shape)
df_vendors_filter_columns[mask]

In [ ]:
# license_text - used for list of foods
vendor_id = '364.1'
df = df_vendors_filter_columns.query('id == @vendor_id')
display(df)

s = df['license_text'].iloc[0]
print(s)

re.split(',\s*(?![^()]*\))', s)

In [ ]:
# license_text - used for list of foods
vendor_id = '809.1'
df = df_vendors_filter_columns.query('id == @vendor_id')
display(df)

s = df['license_text'].iloc[0]
print(s)

re.split(',\s*(?![^()]*\))', s)

# Drinks

In [ ]:
s = df_vendors_filter_columns['alcoholic_beverages'].str.len().value_counts()
display(s)

In [ ]:
s = df_vendors_filter_columns['alcoholic_beverages'].str.len()
df_drink_vendors = df_vendors_filter_columns[s!=0]

print(df_drink_vendors.shape)
display(df_drink_vendors.head())

In [ ]:
# of locations that serve alcoholic beverages
len(df_drink_vendors['name'].unique())

In [ ]:
alcoholic_beverages = df_drink_vendors['alcoholic_beverages'].str.split('\x0b').explode()
print(len(alcoholic_beverages.unique()))

alcoholic_beverages.sample(25)

In [ ]:
alcoholic_beverages[alcoholic_beverages.str.startswith('Big Wood Chocolate Chip Cookie Beer')]

In [ ]:
alcoholic_beverages[alcoholic_beverages.str.startswith('Castle')]

In [ ]:
df_drinks = None
df_drinks = (
    df_vendors_filter_columns[df_vendors_filter_columns['alcoholic_beverages'].str.len() != 0]

    .assign(drinkName      = lambda df_:df_['alcoholic_beverages'].str.split('\x0b')).explode('drinkName')
    .assign(isNew          = lambda df_:df_['drinkName'].str.contains('NEW'))
    .assign(isOnlyAtFair   = lambda df_:df_['drinkName'].str.contains('ONLY AT THE FAIR'))

    .assign(drinkName       = lambda df_:df_['drinkName'].str.replace('(NEW)', '', regex=False))
    .assign(drinkName       = lambda df_:df_['drinkName'].str.replace('(ONLY AT THE FAIR)', '', regex=False))
    .assign(drinkName       = lambda df_:df_['drinkName'].str.replace('’', "'", regex=False))
             
    .assign(drinkName       = lambda df_:df_['drinkName'].str.strip())
             
    .drop(columns=['alcoholic_beverages', 'license_text', 'keywords', 'is_food', 'categories'])
)

drinks_to_exclude = []
drinks_to_exclude += ['Blue Moon', 'Bud Light', 'Budweiser', 'Busch Light', 'Coors Light', 'Corona']
drinks_to_exclude += ['Dos Equis', 'Pabst Blue Ribbon', 'Pacifico', 'Heineken', 'Guinness', 'Stella Artois', 'Miller Lite', 'Michelob Golden Draft Light']
cond1 = df_drinks['isNew'] == True
cond2 = df_drinks['isOnlyAtFair'] == True
cond3 = df_drinks.loc[:, 'drinkName'].str.contains(r'\b(?:{})\b'.format('|'.join(drinks_to_exclude)))
df_drinks = df_drinks[cond1 | cond2 | ~cond3] 
df_drinks = df_drinks.merge(df_drinks.groupby('drinkName')['id'].agg(vendorIDs = list).reset_index(), how='left', on='drinkName')


with open('./json/db.json', 'w') as json_file:
    
    # ---------- DRINKS ----------
    df1 = (df_drinks
           .drop_duplicates(subset='drinkName')[['drinkName', 'isNew', 'isOnlyAtFair', 'vendorIDs']]
           .reset_index()
           .rename(columns={'index': 'id', 'drinkName': 'name'})
           .assign(id = lambda df_:df_['id'].add(1000).astype(str))
          )
    #df1.to_json('./json/drinks.json', orient='records')
    
    # ---------- FOODS ----------
    df2 = (df_new_vendors[['vendor_id', 'name', 'excerpt', 'description']]
           .reset_index()
           .assign(vendor_id = lambda df_:df_['vendor_id'].str.split())
           .rename(columns={'index': 'id', 'vendor_id': 'vendorIDs'})
           .assign(id = lambda df_:df_['id'].add(2000).astype(str))
        )
    
    # ---------- VENDORS ----------
    df3 = df_vendors_filter_columns[['id', 'name', 'directions', 'latitude', 'longitude']]
    #df3.to_json('./json/drink_vendors.json', orient='records')

    
    df_item_to_vendors = pd.concat([df1, df2])[['id', 'vendorIDs']]
    df_vendor_to_items = df_item_to_vendors.explode('vendorIDs').groupby('vendorIDs')['id'].agg(itemIDs=list).reset_index().rename(columns={'vendorIDs':'id'})

    df1 = df1.drop(columns=['vendorIDs'])
    df2 = df2.drop(columns=['vendorIDs'])

    js = {
            'drinks'        : json.loads(df1.to_json(orient='records')),
            'foods'         : json.loads(df2.to_json(orient='records')),
            'vendors'       : json.loads(df3.to_json(orient='records')),
            'itemToVendors' : json.loads(df_item_to_vendors.to_json(orient='records')),
            'vendorToItems' : json.loads(df_vendor_to_items.to_json(orient='records'))
         }
    json.dump(js, json_file, indent=4)

In [ ]:
df_item_to_vendors.head()

In [ ]:
#df_drinks = df_drinks.merge(df_drinks.groupby('drinkName')['id'].agg(vendorIDs = list).reset_index(), how='left', on='drinkName')

df_item_to_vendors.explode('vendorIDs').groupby('vendorIDs')['itemID'].agg(itemIDs=list).reset_index().rename(columns={'vendorIDs':'vendorID')
                                                                                                    
                                                                                            

In [ ]:
df = (
    df_item_to_vendors.head()
    .assign(vendorID= lambda df_:df_['vendorIDs'].explode('vendorID'))
)
df

In [ ]:
df_drinks = None
df_drinks = (
    df_vendors_filter_columns[df_vendors_filter_columns['alcoholic_beverages'].str.len() != 0].head()

     .assign(drinkName      = lambda df_:df_['alcoholic_beverages'].str.split('\x0b')).explode('drinkName')
#     .assign(isNew          = lambda df_:df_['drinkName'].str.contains('NEW'))
#     .assign(isOnlyAtFair   = lambda df_:df_['drinkName'].str.contains('ONLY AT THE FAIR'))

#     .assign(drinkName       = lambda df_:df_['drinkName'].str.replace('(NEW)', '', regex=False))
#     .assign(drinkName       = lambda df_:df_['drinkName'].str.replace('(ONLY AT THE FAIR)', '', regex=False))
#     .assign(drinkName       = lambda df_:df_['drinkName'].str.replace('’', "'", regex=False))
             
#     .assign(drinkName       = lambda df_:df_['drinkName'].str.strip())
             
#     .drop(columns=['alcoholic_beverages', 'license_text', 'keywords', 'is_food', 'categories'])
)
df_drinks

In [ ]:
df1.head() 

In [ ]:
df2.head()

In [ ]:
df3.head()

In [ ]:
df_vendors_filter_columns.head()

In [ ]:
df_new_vendors.head()

In [ ]:
df_drinks.head()

In [ ]:
df_drinks.merge(df_drinks.groupby('drinkName')['id'].agg(vendor_ids = list).reset_index(), how='left', on='drinkName')

In [ ]:
df_drinks.sample(20)

In [ ]:
df_drinks['drinkName'].value_counts().head()

In [ ]:
df_drinks.query('drinkName == "Grain Belt Premium"')

In [ ]:
df_drinks.groupby('drinkName')['id'].agg(vendor_ids=list)#.reset_index().query('drinkName == "Grain Belt Premium"')

In [ ]:
# Watch out for trailing spaces
df_drinks.query('drinkName == "Castle Danger Orange Cream Ale"')

In [ ]:
# Watch out for trailing spaces - this should be fixed by includeing the space when replacing (NEW) and (ONLY AT THE FAIR)
df_drinks.query('drinkName == "Castle Danger Orange Cream Ale "')

In [ ]:
df_drinks.query('drinkName == "Big Wood Chocolate Chip Cookie Beer"')

In [ ]:
df_drinks.query('drinkName == "Big Wood Chocolate Chip Cookie Beer "')

In [ ]:
cond = df_drinks['drinkName'].str.startswith('Castle')
df_drinks[cond]

In [ ]:
df_drinks[['isNew', 'isOnlyAtFair']].sum()

In [ ]:
# https://www.mnstatefair.org/general-info/specialty-brews-and-beverages/

In [ ]:
df_drinks[df_drinks['isOnlyAtFair']].drop_duplicates('drinkName')['drinkName'].sort_values()

In [ ]:
df_drinks[df_drinks['isNew']].drop_duplicates('drinkName')['drinkName'].sort_values()

In [ ]:
df_drinks.query('(isNew ==True)').drop_duplicates(subset='drinkName').sort_values('drinkName')

# Foods

In [ ]:
exclude = ['bottled water', 'Diet Coke', 'Coca-Cola', 'Sprite', 'Pepsi', 'Diet Pepsi', 'Milk', 'Coffee', 'Beer', 'Root Beer', 'Sierra Mist', 'Dr Pepper', 'Iced Tea', 'Mountain Dew', 'Mello Yello',
           'Diet Mountain Dew', 'Powerade', 'Orange Juice', 'Pink Lemonade', 'Cherry Coke', 'Lemonade', 'Fresh-squeezed lemonade', '7-UP', 'Fanta Orange', 'Barq\'s Root Beer',
           'Orange soda', 'Hot Chocolate', '', 'Gatorade', 'Orange Drink', 'Juice', 'Mug Root Beer', 'Chocolate Milk', 'Brewed malt beverages', 'Glow sticks', 'Cheese corn',
           'Bacon Cheeseburgers', 'Hamburgers', 'Hot dogs', 'ice cream cones', 'Foot long hot dogs', 'Caramel corn', 'root beer floats', 'Floats', 'Orange floats', 'Nachos',
           'popcorn', 'Cheeseburgers', 'French Fries', 'sundaes', 'Ice Cream', 'Minnesota wine', 'Grape soda', 'Eggs', 'Lattes', 'Corn Dogs', 'Corn dogs - 12"', 'espresso', 'Sausage',
           'Sparkling water', 'Cappuccino', 'Fruit Punch', 'BBQ chips', 'dinners (beans, coleslaw)', 'Iced coffee', "Schell's beer", 'Red Bull slushes', 'honey sriracha sauce'
          ]
df_foods = (
    df_vendors_filter_columns.query('is_food != 0')
    #df_vendors_filter_columns.query('is_food == 2')
    .assign(food=     lambda df_:df_['license_text'].str.split(',s*(?![^()]*\))')).explode('food')  
    .assign(food=     lambda df_:df_['food'].str.strip())
    #.drop(columns=['alcoholic_beverages'])
    .drop(columns=['license_text', 'keywords', 'is_food'])
    #.drop(columns=['categories'])
)

cond = df_foods['categories'].apply(lambda x: '41' in x)
df_foods = df_foods[cond]

cond = ~df_foods['food'].isin(exclude)
df_foods = df_foods[cond]

print(df_foods.shape)    
    
#['food'].str.strip() #.value_counts().drop(labels=exclude).head(50)
#foods= df_food.assign(food=df_food['license_text'].str.split(',s*(?![^()]*\))')).explode('food')['food'].str.strip().reset_index(drop=True)
#s = foods.str.len().sort_values(ascending=False).index
#foods.reindex(s).head(250).sort_values()

df_foods.sample(20)

In [ ]:
cond1 = df_vendors_filter_columns['is_food'] != 0
cond2 = df_vendors_filter_columns['categories'].apply(lambda x: '41' in x)
df_new_food = df_vendors_filter_columns[cond1 & cond2]
s = df_new_food.assign(food=df_food['license_text'].str.split(',s*(?![^()]*\))')).explode('food')['food'].str.strip().value_counts()#.drop(labels=exclude)
#s[s==1].head(50)

In [ ]:
df_vendors_filter_columns.sort_values('name').iloc[0]['name']

In [ ]:
df.query('name == "Andy\'s Grille"')['alcoholic_beverages'].iloc[0].split('\x0b')

In [ ]:
name = "LuLu's Public House"
df_vendors.query('name == @name')['alcoholic_beverages'].iloc[0].split('\x0b')

In [ ]:
s = df_vendors.query('name == @name')['license_text'].iloc[0]
re.split(',\s*(?![^()]*\))', s)

In [ ]:
name = 'Solem Concessions Cheese Curds and Mini Donuts'
s = df_vendors.query('name == @name')['license_text'].iloc[0] 
#s.split(', ')  

re.split(',\s*(?![^()]*\))', s)

# don't split on commas within ()  !!

In [ ]:
df_vendors.query('name == @name')#.loc[:,['latitude', 'longitude']]

In [ ]:
df_vendors['license_text'] = df_vendors['license_text'].fillna('')
cond = df_vendors['license_text'].str.contains('donut')
df_vendors[cond]

In [ ]:
df_vendors['new_this_year'].unique()

In [ ]:
df_vendors['alcoholic_beverages'] = df_vendors['alcoholic_beverages'].fillna('')

cond = df_vendors['alcoholic_beverages'].str.contains('NEW')
df_vendors[cond]

In [ ]:
names = ['The Blue Barn', 'LuLu\'s Public House', 'Fluffy\'s Hand Cut Donuts', 'Minnesota Wine Country', 'O\'Gara\'s at the Fair', 'Pronto Pups', 'The Hangar', 'Turkey to Go']
markers = df_vendors.query('name in @names').loc[:,['name', 'latitude', 'longitude']]

In [ ]:
colors = ['red', 'blue', 'green', 'purple', 'orange', 'pink',
          'darkred', 'darkblue', 'darkgreen', 'darkpurple',
          'lightred', 'lightgray',
          'beige', 'cadetblue', 'white', 'gray', 'black' ]

In [ ]:
#https://getbootstrap.com/docs/3.3/components/

import folium
m = folium.Map(location=[44.98106, -93.174351], zoom_start=17)

for marker in markers.itertuples():
    color = colors[names.index(marker.name)]
    folium.Marker(
        location=[marker.latitude, marker.longitude],
        popup=marker.name,
        icon=folium.Icon(color=color, icon="star-empty"),    #  heart  glass  star  cutlery  star   music     tar-empty
    ).add_to(m)

m

In [ ]:
# (NEW)  (ONLY AT THE FAIR)  Seltzer
df_drinks['is_new'] = df_drinks['drink'].str.contains('NEW')
df_drinks['is_only_at_fair'] = df_drinks['drink'].str.contains('ONLY AT THE FAIR')
df_drinks['is_seltzer'] = df_drinks['drink'].str.contains('Seltzer')

df_drinks['drink'] = df_drinks['drink'].str.replace('(NEW)', '', regex=False)
df_drinks['drink'] = df_drinks['drink'].str.replace('(ONLY AT THE FAIR)', '', regex=False)
df_drinks['drink'] = df_drinks['drink'].str.strip()

print(df_drinks.shape)
df_drinks.head()

df_drinks = df_drinks.drop(columns=['alcoholic_beverages', 'license_text', 'keywords', 'is_food', 'categories'])

print(df_drinks.shape)
#df_drinks.head()
df_drinks.sample(50)
